# 1st

## Model Explore

In [ ]:
!pip install -q transformers accelerate bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.9 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
notebook_login()

## more model

In [ ]:
torch.cuda.is_available()

True

In [ ]:
model_id = "meta-llama/Llama-2-7b-hf"

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Load model with 8-bit precision (needs less memory)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True,  # For lower RAM usage (needs `bitsandbytes`)
    torch_dtype=torch.float16,
)


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
def answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    # output = model.generate(**inputs, max_new_tokens=200)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        eos_token_id=tokenizer.eos_token_id)
    # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
prompt = " I have knee pain which medicine should i take?"
answer(prompt)

' I have knee pain which medicine should i take?\nI have knee pain which medicine should i take?\nI have knee pain which medicine should i take? I am 61 and I am in pain.\nThe knee is a very complex joint. Pain in the knee can be due to a number of problems, including arthritis, meniscus tear, ligament tear, bone bruise, bursitis, gout, tendonitis, and many more.\nIf your knee pain is due to a problem in the joint itself, your doctor will want to perform a physical examination and may also order an x-ray to see if there is a problem in the bones.\nIf the problem is due to'

In [ ]:
prompt = "I have headache which medicine should i take?"
answer(prompt)

'I have headache which medicine should i take?\nMost headaches are caused by tension, sinusitis, or a virus. The first thing you should do is try to relax. Take a warm bath, or apply a hot compress to the head. You can also try taking an over-the-counter pain reliever like ibuprofen or acetaminophen.\nIf the headache is severe or if you have a fever, you should see a doctor. A doctor can prescribe medication to help with the pain.\nMost headaches are caused by tension, sinusitis, or a virus. The first thing you should do is try to relax. Take a warm bath, or apply a hot compress to the head. You'

# detection

In [ ]:
# install embedding library
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer, util
import torch

In [ ]:
import json

# Load data from benchmarking.json
with open('/content/benchmarking.json', 'r') as f:
    data = json.load(f)

# Use only the top 40 questions
data = data[:40]

In [ ]:
data

[{'question': 'What are common causes of headaches?',
  'answer': 'Common causes include stress, dehydration, poor posture, lack of sleep, and visual strain.'},
 {'question': 'How can you differentiate between a tension headache and a migraine?',
  'answer': 'Tension headaches usually cause mild to moderate diffuse pain around the head. Migraines are often more severe, with pulsating pain on one side of the head, and can include nausea and light sensitivity.'},
 {'question': 'What home remedies are effective for relieving headaches?',
  'answer': 'Effective remedies include hydration, rest, applying a cold or warm compress to the head, and practicing stress-reduction techniques such as meditation or gentle stretching.'},
 {'question': 'What are typical causes of lower back pain?',
  'answer': 'Common causes include muscle or ligament strain, poor posture, heavy lifting, and underlying medical conditions like arthritis or disk disease.'},
 {'question': 'Describe simple stretches to alle

In [ ]:
# 2. Load SBERT embedder onto GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedder = SentenceTransformer("all-MiniLM-L6-v2").to(device)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# --- 3. Define sampling settings (seed + temperature) ---
N = 5
seeds = [42, 43, 44, 45, 46]
temps = [0.5, 0.6, 0.7, 0.8, 0.9]
settings = list(zip(seeds, temps))
all_avg_scores = []
# --- 4. Loop through each Q&A pair ---
for item in data:
    q  = item["question"]
    ga = item["answer"]

    # 4a. Generate N answers with different seeds & temperatures
    samples = []
    for seed, temp in settings:
        torch.manual_seed(seed)
        prompt_text = f"Question: {q}\nAnswer:"
        inputs = tokenizer(prompt_text, return_tensors="pt").to(device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=temp,
            top_p=0.9,
            top_k=50,
            eos_token_id=tokenizer.eos_token_id
        )
        ans = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        samples.append(ans)

    # 4b. Embed the gold answer and the samples
    gold_emb   = embedder.encode(ga,       convert_to_tensor=True).to(device)
    sample_emb = embedder.encode(samples,  convert_to_tensor=True).to(device)

    # 4c. Compute cosine similarities and average
    sims = util.cos_sim(gold_emb, sample_emb)[0]
    avg_score = sims.mean().item()
    all_avg_scores.append(avg_score)

    # 4d. Print the results
    print("-" * 80)
    print(f"Q: {q}")
    print(f"Gold: {ga!r}")
    print(f" Average semantic similarity: {avg_score:.3f}")
    for i, (score, ans) in enumerate(zip(sims, samples), 1):
        print(f"   {i:>2}. [{temps[i-1]:.1f} temp, seed={seeds[i-1]}] {score.item():.3f} → {ans}")
    print("-" * 80)

--------------------------------------------------------------------------------
Q: What are common causes of headaches?
Gold: 'Common causes include stress, dehydration, poor posture, lack of sleep, and visual strain.'
 Average semantic similarity: 0.385
    1. [0.5 temp, seed=42] 0.388 → Question: What are common causes of headaches?
Answer: There are many causes of headaches. The most common are tension headaches, migraines, and cluster headaches. Tension headaches are the most common type of headache. They are usually described as a dull, constant pain that feels like a tight band around the head. Migraines are more severe headaches that are often accompanied by nausea, vomiting, and sensitivity to light and sound. Cluster headaches are a type of headache that occurs in groups, usually on one side of the head. They are often described as a sharp, piercing pain.
Question: What are some common triggers for headaches?
Answer: There are many triggers for headaches. The most common are


In [ ]:

overall_avg = sum(all_avg_scores) / len(all_avg_scores)
print(f"\n=== Overall Average Across {len(all_avg_scores)} Questions: {overall_avg:.3f} ===")

=== Overall Average Across 60 Questions: 0.289 ===
